# Содержание

* [1 Загрузка данных](#1-Загрузка-данных)
* [2 Формирование таблицы рейтинга](#2-Формирование-таблицы-рейтинга)
* [3 Построение рекомендательной системы](#3-Построение-рекомендательной-системы)
* [4 Общий вывод](#4-Общий-вывод)

---

# Этап 3. Рекомендательная система

**Задача:** Формирование системы рекомендаций.

**Источник данных:** `data/preprocessed_data/data_train.csv`, `data/preprocessed_data/data_test.csv`.

**Характер данных:** проанализированные данные о заказах в Сингапуре.

---

## 1 Загрузка данных

Установка библиотек:

In [1]:
%%capture no-display
!pip install geopy geohash2

Импорт библиотек:

In [2]:
import numpy as np
import pandas as pd

import os
import re

from typing import Tuple, List
from datetime import datetime

import requests
import geohash2

from geopy.distance import geodesic

import pickle

from tensorflow.keras.models import load_model
from tensorflow.keras.layers import TextVectorization

---

Задание пути до папки с данными:

In [3]:
PATH = 'data/preprocessed_data/'

Выведение на экран содержимого папки с данными:

In [4]:
os.listdir(PATH)

['data_test.csv',
 'data_test_cuisine.csv',
 'data_train.csv',
 'data_train_cuisine.csv']

Загрузка наборов данных:

In [5]:
join_path = lambda x: os.path.join(PATH, x)

data_train = pd.read_csv(join_path('data_train.csv'), index_col=0)
data_test = pd.read_csv(join_path('data_test.csv'), index_col=0)

>* Поскольку модель обучена на обучающей выборке и протестирована на тестовой, в задаче построения рекомендательной системы эти данные можно объединить.

Объединение данных:

In [6]:
data = pd.concat([data_train, data_test]).reset_index(drop=True)

Выведение на экран первых строк набора данных:

In [7]:
display(data.head(3))

print('Data shape:', data.shape)

,name,vendor_lat_lon,hour,time_of_day,primary_cuisine
0,chicken cutlet with rice,"(1.3, 104.0)",19,evening,western
1,chicken cutlet with rice,"(1.3, 104.0)",18,evening,western
2,chicken chop,"(1.3, 104.0)",19,evening,western


Data shape: (1162920, 5)


>**Вывод**
>
>* Поскольку при загрузке данных значения типа `object` преобразовываются к строке, необходимо преобразовать их значения к числовому формату.

---

## 2 Формирование таблицы рейтинга

Задание функции, преобразующей тип значений координат:

In [8]:
def decode_hash(code: str) -> Tuple[float]:
    '''Decode geo hash to latitude and longitude.
    
    Args:
      - code (str) - hash code
    '''
    code = re.sub('[,\(\)]', '', code).split()
    
    return (float(code[0]), float(code[1]))

---

Преобразование типа значений координат:

In [9]:
data['vendor_lat_lon'] = data['vendor_lat_lon'].apply(decode_hash)

>* Для формирования рейтинга ресторанов необходимо произвести группировку значений набора данных по следующим столбцам: `primary_cuisine`, `name`, `time_of_day`, `vendor_lat_lon` и отсортировать значения по уменьшению числа заказов.

Группировка данных:

In [10]:
data_count_rating = pd.DataFrame(
    data.groupby(
        ['primary_cuisine', 'name', 'time_of_day', 'vendor_lat_lon']
    )['primary_cuisine'].value_counts()
).sort_values(
    by='count', 
    ascending=False
).reset_index()

Выведение на экран первых строк набора данных для проверки применённых изменений:

In [11]:
data_count_rating.head(3)

,primary_cuisine,name,time_of_day,vendor_lat_lon,count
0,chinese,minced meat tofu rice,evening,"(1.4, 103.8)",16
1,mala xiang guo,taiwan sausage,evening,"(1.5, 103.8)",16
2,american,chocolate pie,evening,"(1.4, 103.9)",16


>* Для сохранения значений координат в удобном для загрузки виде необходимо разделить их на два отдельных столбца: широту и долготу.

Разъединение координат на широту и долготу:

In [12]:
data_count_rating = data_count_rating.assign(
    **pd.DataFrame(
        data_count_rating['vendor_lat_lon'].tolist(), 
        columns=['vendor_lat','vendor_lon']
    )
)

Удаление столбца:

In [13]:
data_count_rating = data_count_rating.drop('vendor_lat_lon', axis=1)

Выведение на экран первых строк набора данных для проверки применённых изменений:

In [14]:
data.head(3)

,name,vendor_lat_lon,hour,time_of_day,primary_cuisine
0,chicken cutlet with rice,"(1.3, 104.0)",19,evening,western
1,chicken cutlet with rice,"(1.3, 104.0)",18,evening,western
2,chicken chop,"(1.3, 104.0)",19,evening,western


---

Сохранение таблицы с рейтингом:

In [15]:
data_count_rating.to_csv('webapp/data/data_count_rating.csv')

---

## 3 Построение рекомендательной системы

**Загрузка моделей и рейтинга**

Загрузка моделей:

In [16]:
multilabel = pickle.load(open('webapp/model/multilabel.pkl', 'rb'))
model = load_model('webapp/model/model.h5')
vectorizer_file = pickle.load(open('webapp/model/vectorizer.pkl', 'rb'))

Загрузка набора данных:

In [17]:
data_count_rating = pd.read_csv('webapp/data/data_count_rating.csv')

---

**Имитация запроса пользователя**

Задание функции, категоризирующей час по времени суток:

In [18]:
def check_daytime(hour: int) -> str:
    '''Convert hour to category.
    
    Args:
      - hour (int) - hour value
    '''
    
    if 4 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 16:
        return 'day'
    elif 16 <= hour < 24:
        return 'evening'
    else:
        return 'night'

Задание функции, трансформирующей предсказания в слова:

In [19]:
def transform_label(label: int, vocab):
    '''Convert label to word.
    
    Args:
      - label (int) - label
      - vocab - vocabulatory
    '''
    
    return vocab.inverse_transform(np.array([label]))

Задание функции, трансформирующей вероятности предсказаний в значения 0 и 1:

In [20]:
def transform_probs_to_labels(probs: np.array[float], threshold: float = 0.5):
    '''Convert probabilities values to labels.
    
    Args:
      - probs (np.array[float]) - prediction probabilities
      - threshold (float, optional) - threshold to convert values. Defaults to 0.5
    '''
    
    return (probs > threshold).astype(int)

TypeError: 'builtin_function_or_method' object is not subscriptable

Задание функции, преобразующей значения предсказаний:

In [ ]:
def preprocess_prediction(pred: List[int], vocab):
    '''Convert labels into words.
    
    Args:
      - pred (List[int]) - prediction labels
      - vocab - vocabulatory
    '''
    
    if sum(pred) == 0:
        pred = 'No results'
    else:
        pred = transform_label(pred, vocab)
        pred = ', '.join(*pred)
    
    return pred

Задание пайплайна обработки входящих значений для получения предсказаний:

In [ ]:
def get_predictions(value: List[float]):
    '''Pipeline to convert input value and output value.
    
    Args:
      - value (List[float]) - predicted values
    '''
    
    # convert vectorizer
    vectorizer = TextVectorization.from_config(vectorizer_file['config'])
    vectorizer.adapt([value])
    vectorizer.set_weights(vectorizer_file['weights'])

    # make predictions
    prediction = model.predict(vectorizer([value]))
    # convert predictions
    prediction = transform_probs_to_labels(prediction)[0]
    # preprocess output
    prediction = preprocess_prediction(prediction, multilabel)

    return prediction

Задание функции, преобразующей адрес пользователя в широту и долготу:

In [ ]:
def convert_place_to_lat_lon(place: str) -> Tuple[float, float]:
    '''Get latitude and longitude through address name.
    
    Args:
      - place (str)
    '''
    
    url = f'https://nominatim.openstreetmap.org/search.php?q={place}&format=jsonv2'
    
    try:
        result = requests.get(url=url)
        result_json = result.json()[0]
        
        return result_json['lat'], result_json['lon']
        
    except:
        return None

Задание функции, вычисляющей расстояния от пользователя до ресторана:

In [ ]:
def calculate_distance(user: Tuple[float], row) -> float:
    '''Calculate distance from customer to vendor.
    
    Args:
      - user (Tuple[float]) - user address latitude and longitude
      - row - row of a dataframe
    '''
    vendor = (row['vendor_lat'], row['vendor_lon'])
    
    return round(geodesic(user, vendor).km, 2)

Задание функции, формирующей рейтинг рекомендаций:

In [ ]:
def create_rating(df: pd.DataFrame(), 
                  place: str, 
                  hour: str, 
                  pred: List[str], 
                  top: int = 5) -> Tuple[str, str]:
    
    '''Filter and sort values to get best results.
    
    Args:
      - df (pd.DataFrame) - dataframe
      - place (str) - user address
      - hour (int) - hout value
      - pred (List[str]) - predicted cuisines
      - top (int, optional) - number of top lines to use. Defaults to 5
    '''
    
    daytime = check_daytime(int(hour))
    user_lat_lon = convert_place_to_lat_lon(place)
    
    # calculate distance between user and vendor
    rating_result = df[df['time_of_day'] == daytime].copy()
    
    rating_result['distance'] = rating_result.apply(
        lambda x: calculate_distance(user_lat_lon, x),
        axis=1
    )
    
    rating_result = rating_result.sort_values(by='distance')
    
    
    # create rating    
    rating = pd.DataFrame()

    for p in pred.split(', '):
        rating = pd.concat([rating, rating_result[rating_result['primary_cuisine'] == p][:top]])

    rating_top = rating.sort_values(by='count', ascending=False)[:top]
    
    rating_top['vendor_lat_lon'] = rating.apply(lambda x: (x['vendor_lat'], x['vendor_lon']), axis=1)
    
    cuisines = list(rating_top['primary_cuisine'].unique())
    locations = list(rating_top['vendor_lat_lon'].unique())

    cuisines = ', '.join(cuisines)
    locations = ', '.join([str(l) for l in locations])

    if cuisines == '' or locations == '':
        cuisines = 'No results'
        locations = 'No results'
    
    return cuisines, locations

---

>**Рекомендательная система**
>
>* Входящие значения:
>    * Название блюда.
>    * Текущий час.
>    * Местоположение пользователя.
>
>* Внутренняя фильтрация и сортировка:
>    * Отфильтрованная повремени суток таблица рейтинга.
>    * ТОП строк отфильтрованной по каждому предсказанному виду кухни таблица рейтинга.
>
>* Результат в виде рекомендаций:
>    * ТОП строк итоговой таблицы, отсортированной по убыванию частоты заказов блюд.

Получение предсказаний по запросу:

In [ ]:
request = 'chicken'
prediction = get_predictions(request)

print('\n', request, '->', prediction)

Формирование рейтинга на основе предсказаний и входящих данных:

In [ ]:
best_result = create_rating(data_count_rating, user_lat_lon, 19, prediction)

Выведение на экран рекомендаций:

In [ ]:
print('Best places primary cuisine:', best_result[0])
print('Best places coords:', best_result[1])

---

## 4 Общий вывод

В ходе формирования рекомендательной системы:

* Из полученных после анализа и отбора данных таблиц была сформирована таблица рейтинга.
* Прописана функция, применяющая к таблице рейтинга полученные предсказания для формирования рекомендаций.

<div style="text-align: center; font-size: 20px; padding: 15px 0;">
    <a href="#Содержание" data-toc-modified-id="Содержание" style="text-decoration: none; color: #296eaa; border: 2px dashed #296eaa; opacity: 0.8; border-radius: 3px; padding: 10px 80px;">
        В начало файла ↑
    </a>
</div>